In [1]:
from haversine import Unit
import haversine as hs
import pandas as pd
import sys
from meteostat import Point, Daily
import matplotlib.pyplot as plt

In [2]:
sys.path.append('../utils/')
from data_cleaning import DataCleaning
from data_extraction import DataExtraction
from feature_extraction import FeatureExtraction

In [3]:
dclean = DataCleaning()
dextract = DataExtraction()
fextract = FeatureExtraction()

In [4]:
clean_df = dextract.load_data('../data/clean_data.csv')
clean_df.tail()

,id,order_id,driver_id,driver_action,lat,lng,driver_location,trip_origin,trip_destination,trip_Start_time,trip_end_time
1557734,1557736,517948,243774,rejected,6.469036,3.566877,"6.469036,3.5668767","6.443373599999999,3.5523123","6.500608199999999,3.5981937",2021-09-10 11:47:26,2021-09-10 13:30:13
1557735,1557737,517948,245447,rejected,6.425431,3.527358,"6.4254311,3.5273581","6.443373599999999,3.5523123","6.500608199999999,3.5981937",2021-09-10 11:47:26,2021-09-10 13:30:13
1557736,1557738,517948,239866,rejected,6.440013,3.525378,"6.4400128,3.5253785","6.443373599999999,3.5523123","6.500608199999999,3.5981937",2021-09-10 11:47:26,2021-09-10 13:30:13
1557737,1557739,517948,243774,rejected,6.469036,3.566877,"6.469036,3.5668767","6.443373599999999,3.5523123","6.500608199999999,3.5981937",2021-09-10 11:47:26,2021-09-10 13:30:13
1557738,1557740,517948,244170,rejected,6.469390,3.572079,"6.4693899,3.5720791","6.443373599999999,3.5523123","6.500608199999999,3.5981937",2021-09-10 11:47:26,2021-09-10 13:30:13


In [5]:
clean_df = clean_df.dropna()
clean_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1555717 entries, 0 to 1557738
Data columns (total 11 columns):
 #   Column            Non-Null Count    Dtype  
---  ------            --------------    -----  
 0   id                1555717 non-null  int64  
 1   order_id          1555717 non-null  int64  
 2   driver_id         1555717 non-null  int64  
 3   driver_action     1555717 non-null  object 
 4   lat               1555717 non-null  float64
 5   lng               1555717 non-null  float64
 6   driver_location   1555717 non-null  object 
 7   trip_origin       1555717 non-null  object 
 8   trip_destination  1555717 non-null  object 
 9   trip_Start_time   1555717 non-null  object 
 10  trip_end_time     1555717 non-null  object 
dtypes: float64(2), int64(3), object(6)
memory usage: 142.4+ MB


# Distance Calculation

In [6]:
starting=clean_df['trip_origin'].to_list()
ending=clean_df['trip_destination'].to_list()


In [9]:
start=clean_df['trip_origin'].to_list()
end=clean_df['driver_location'].to_list()

In [13]:
from geopy import distance

def calculate_distances(starting_coordinates, ending_coordinates):
    calculated_distances = []
    for i in range(len(starting_coordinates)):
        # Extract and convert coordinates from string representations to floats
        start_lat, start_lon = map(float, str(starting_coordinates[i]).strip('()').split(','))
        end_lat, end_lon = map(float, str(ending_coordinates[i]).strip('()').split(','))

        # Calculate distance using geopy
        start_coords = (start_lat, start_lon)
        end_coords = (end_lat, end_lon)
        calculated_distance = distance.distance(start_coords, end_coords).km

        calculated_distances.append(calculated_distance)

    return calculated_distances


In [14]:
clean_df['trip_distance']=calculate_distances(starting,ending)
clean_df['trip_to_driver_distance']=calculate_distances(start,end)

In [15]:
clean_df.head()

,id,order_id,driver_id,driver_action,lat,lng,driver_location,trip_origin,trip_destination,trip_Start_time,trip_end_time,trip_distance,trip_to_driver_distance
0,1,392001,243828,accepted,6.602207,3.270465,"6.6022066,3.2704649","6.6010417,3.2766339","6.4501069,3.3916154",2021-07-01 09:30:59,2021-07-01 09:34:36,20.984319,0.694264
1,2,392001,243588,rejected,6.592097,3.287445,"6.5920972,3.2874447","6.6010417,3.2766339","6.4501069,3.3916154",2021-07-01 09:30:59,2021-07-01 09:34:36,20.984319,1.551694
2,3,392001,243830,rejected,6.596133,3.281784,"6.5961334,3.2817841","6.6010417,3.2766339","6.4501069,3.3916154",2021-07-01 09:30:59,2021-07-01 09:34:36,20.984319,0.786777
3,4,392001,243539,rejected,6.596142,3.280526,"6.5961416,3.2805263","6.6010417,3.2766339","6.4501069,3.3916154",2021-07-01 09:30:59,2021-07-01 09:34:36,20.984319,0.692054
4,5,392001,171653,rejected,6.609232,3.288800,"6.6092317,3.2887999","6.6010417,3.2766339","6.4501069,3.3916154",2021-07-01 09:30:59,2021-07-01 09:34:36,20.984319,1.621848


In [16]:
# Convert the columns to datetime
clean_df['trip_end_time'] = pd.to_datetime(clean_df['trip_end_time'])
clean_df['trip_Start_time'] = pd.to_datetime(clean_df['trip_Start_time'])

In [17]:
clean_df['trip_time'] = (clean_df.trip_end_time - clean_df.trip_Start_time)
clean_df['trip_speed'] = clean_df['trip_distance'] / clean_df['trip_time'].dt.total_seconds()
clean_df[['trip_to_driver_distance', 'trip_distance','trip_time','trip_speed']].head()

,trip_to_driver_distance,trip_distance,trip_time,trip_speed
0,0.694264,20.984319,0 days 00:03:37,0.096702
1,1.551694,20.984319,0 days 00:03:37,0.096702
2,0.786777,20.984319,0 days 00:03:37,0.096702
3,0.692054,20.984319,0 days 00:03:37,0.096702
4,1.621848,20.984319,0 days 00:03:37,0.096702


In [18]:
clean_df.head()

,id,order_id,driver_id,driver_action,lat,lng,driver_location,trip_origin,trip_destination,trip_Start_time,trip_end_time,trip_distance,trip_to_driver_distance,trip_time,trip_speed
0,1,392001,243828,accepted,6.602207,3.270465,"6.6022066,3.2704649","6.6010417,3.2766339","6.4501069,3.3916154",2021-07-01 09:30:59,2021-07-01 09:34:36,20.984319,0.694264,0 days 00:03:37,0.096702
1,2,392001,243588,rejected,6.592097,3.287445,"6.5920972,3.2874447","6.6010417,3.2766339","6.4501069,3.3916154",2021-07-01 09:30:59,2021-07-01 09:34:36,20.984319,1.551694,0 days 00:03:37,0.096702
2,3,392001,243830,rejected,6.596133,3.281784,"6.5961334,3.2817841","6.6010417,3.2766339","6.4501069,3.3916154",2021-07-01 09:30:59,2021-07-01 09:34:36,20.984319,0.786777,0 days 00:03:37,0.096702
3,4,392001,243539,rejected,6.596142,3.280526,"6.5961416,3.2805263","6.6010417,3.2766339","6.4501069,3.3916154",2021-07-01 09:30:59,2021-07-01 09:34:36,20.984319,0.692054,0 days 00:03:37,0.096702
4,5,392001,171653,rejected,6.609232,3.288800,"6.6092317,3.2887999","6.6010417,3.2766339","6.4501069,3.3916154",2021-07-01 09:30:59,2021-07-01 09:34:36,20.984319,1.621848,0 days 00:03:37,0.096702


In [19]:
import datetime
import holidays

ng_holidays = holidays.country_holidays('NG')

def isWeekend(df_date):
    if df_date.weekday() < 5:
        return 0
    else:  # 5 Sat, 6 Sun
        return 1

def isHoliday(df_date):
    try:
        if df_date.date() in ng_holidays:
            return 1
        else:
            return 0
    except Exception as e:
        return 0


In [20]:
clean_df['isHoliday'] = clean_df['trip_Start_time'].apply(lambda x: isHoliday(x))
clean_df['isWeekend'] = clean_df['trip_Start_time'].apply(lambda x: isWeekend(x))

In [21]:
clean_df['isWeekend'].value_counts()

isWeekend
0    1355841
1     199876
Name: count, dtype: int64

In [22]:
clean_df['isHoliday'].value_counts()

isHoliday
0    1555717
Name: count, dtype: int64

In [23]:
clean_df.head()

,id,order_id,driver_id,driver_action,lat,lng,driver_location,trip_origin,trip_destination,trip_Start_time,trip_end_time,trip_distance,trip_to_driver_distance,trip_time,trip_speed,isHoliday,isWeekend
0,1,392001,243828,accepted,6.602207,3.270465,"6.6022066,3.2704649","6.6010417,3.2766339","6.4501069,3.3916154",2021-07-01 09:30:59,2021-07-01 09:34:36,20.984319,0.694264,0 days 00:03:37,0.096702,0,0
1,2,392001,243588,rejected,6.592097,3.287445,"6.5920972,3.2874447","6.6010417,3.2766339","6.4501069,3.3916154",2021-07-01 09:30:59,2021-07-01 09:34:36,20.984319,1.551694,0 days 00:03:37,0.096702,0,0
2,3,392001,243830,rejected,6.596133,3.281784,"6.5961334,3.2817841","6.6010417,3.2766339","6.4501069,3.3916154",2021-07-01 09:30:59,2021-07-01 09:34:36,20.984319,0.786777,0 days 00:03:37,0.096702,0,0
3,4,392001,243539,rejected,6.596142,3.280526,"6.5961416,3.2805263","6.6010417,3.2766339","6.4501069,3.3916154",2021-07-01 09:30:59,2021-07-01 09:34:36,20.984319,0.692054,0 days 00:03:37,0.096702,0,0
4,5,392001,171653,rejected,6.609232,3.288800,"6.6092317,3.2887999","6.6010417,3.2766339","6.4501069,3.3916154",2021-07-01 09:30:59,2021-07-01 09:34:36,20.984319,1.621848,0 days 00:03:37,0.096702,0,0


In [27]:
clean_df.to_csv('../data/features.csv',index=False)

In [25]:
# Define location for Lagos, Nigeria
lagos = Point(6.5244, 3.3792)

# Define time range for the data
start_date = clean_df['trip_Start_time'].min().to_pydatetime()
# decrease one day from the start date
start_date = start_date - pd.Timedelta(days=1)
end_date = clean_df['trip_Start_time'].max().to_pydatetime()
start_date, end_date

(datetime.datetime(2021, 6, 30, 6, 39, 51),
 datetime.datetime(2022, 2, 27, 9, 7, 59))

In [36]:
# Get daily weather data for Lagos
weather_data = Daily(lagos, start_date, end_date)
weather_data = weather_data.fetch()

# Select only the desired columns
weather_data = weather_data[['tavg', 'wdir', 'wspd',  'pres']]

# Display the first few rows of the weather data
print(weather_data.tail())

            tavg   wdir  wspd    pres
time                                 
2022-02-23  29.6  217.0   9.8  1011.8
2022-02-24  28.4  227.0   9.3  1011.4
2022-02-25  29.6  233.0   8.7  1010.5
2022-02-26  29.9  215.0  10.8  1009.9
2022-02-27  28.3  218.0  10.4  1010.6


In [44]:
# Save weather_data to a CSV file
weather_data.to_csv('../data/weather_data.csv', index=False)

In [42]:
clean_df.to_csv('../data/features.csv',index=False)